# 02 - Data from the Web

We start importing all the libraries that we are going to use.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Generating the base of our futures requests

In [2]:
# We will create a string that contains the url to the isacademia form
# From there we will get all the values we need for making our request petitions.
payloadform = {'ww_i_reportModel': '133685247'}
isa_baseurl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS'
isa_formurl= isa_baseurl + ".filter"
r = requests.get(isa_formurl, params=payloadform)
# Here is the url to the isa form
s = r.url + " " + str(r.status_code)  
s

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247 200'

Now we will build a framework that lets us easily get a pandas Dataframe for given parameters.

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')
form = form = soup.find('form')

def findSelectNameAndOptionValue(inputLabel, optionLabel):
    th = form.find('th', string=inputLabel)
    td = th.nextSibling
    select = td.find('select')
    option = select.find('option', string=optionLabel)
    return (select['name'], option['value'])

findSelectNameAndOptionValue("Unité académique", "Informatique")

('ww_x_UNITE_ACAD', '249847')

In [5]:
def generateParams(humanReadableParams):
    fixedParams = {
        'ww_i_reportModel': '133685247',
        'ww_i_reportModelXsl': '133685270',
        'ww_x_GPS': '-1',
    }
    params = dict([findSelectNameAndOptionValue(x, y) for (x, y) in humanReadableParams.items()])
    return {**fixedParams, **params}

# Example
exampleParams = {
    "Unité académique": "Informatique",
    "Période académique": "2016-2017",
    "Période pédagogique": "Bachelor semestre 1",
    "Type de semestre": "Semestre d'automne",
}
generateParams(exampleParams)

{'ww_i_reportModel': '133685247',
 'ww_i_reportModelXsl': '133685270',
 'ww_x_GPS': '-1',
 'ww_x_HIVERETE': '2936286',
 'ww_x_PERIODE_ACAD': '355925344',
 'ww_x_PERIODE_PEDAGO': '249108',
 'ww_x_UNITE_ACAD': '249847'}

In [6]:
# Now we will parse the html using BeautifulSoup library.
# The data comes in a html table. It won't be hard to parse it into pandas dataframe.

# For `Bachelor semestre`, `Master semetre`, `Projet Master` the structure of the table is always the same.

def extractTable(html):
    """Extract the table from HTML into a pandas Dataframe."""
    BStable = BeautifulSoup(html, 'html.parser').find('table')
    BSRows = BStable.findAll('tr')
    # BSrows[0] is just the title of the table, so we ignore it.
    headerBSRow = BSRows[1]
    headers = [th.string for th in headerBSRow]
    # We drop the last column, because it's always empty (originally it is used to make the HTML table look better).
    rows = [[tr.string for tr in BSRow.findAll('td')[:-1]] for BSRow in BSRows[2:]]
    return pd.DataFrame(data=rows, columns=headers)

In [8]:
import random
def getDataFrame(humanReadableParams):
    isa_dataurl = isa_baseurl + ".html"
    r = requests.get(isa_dataurl, params=generateParams(humanReadableParams))
    return extractTable(r.text)
    
getDataFrame(exampleParams).head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,None,None,None,None,None,Présent,None,None,235688
1,Monsieur,Ahn Seongho,None,None,None,None,None,Présent,None,None,274015
2,Madame,Alemanno Sara,None,None,None,None,None,Présent,None,None,268410
3,Monsieur,Althaus Luca,None,None,None,None,None,Présent,None,None,271464
4,Monsieur,Assi Karim,None,None,None,None,None,Présent,None,None,274518
